In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision.datasets import FashionMNIST
from torchvision.transforms import ToTensor

In [2]:
class NormalCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 10, 3)
        self.conv2 = nn.Conv2d(10, 20, 3)
        self.conv3 = nn.Conv2d(20, 30, 3)
        self.conv4 = nn.Conv2d(30, 20, 3)
        self.conv5 = nn.Conv2d(20, 10, 3)

        self.fc1 = nn.Linear(10 * 4 * 4, 50)
        self.fc2 = nn.Linear(50, 10)

        self.pool = nn.MaxPool2d(2)
        
    def forward(self, x):
        # print("Before Calc")
        x = F.relu(self.conv1(x))
        # print("After Conv1", x.size())
        x = F.relu(self.conv2(x))
        # print("After Conv2", x.size())
        x = F.relu(self.conv3(x))
        # print("After Conv3", x.size())
        x = F.relu(self.conv4(x))
        x = self.pool(x)
        # print("After Conv4", x.size())
        x = F.relu(self.conv5(x))
        x = self.pool(x)
        # print("After Conv5", x.size())
        x = x.view(-1, 10 * 4 * 4)
        # print("After dim reduction", x.size())
        x = F.relu(self.fc1(x))
        # print("After fc1", x.size())
        x = self.fc2(x)
        # print("After fc2", x.size())
        return x

cnn = NormalCNN()
output = cnn(torch.rand(1, 1, 28, 28))

In [3]:
class SepCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Sequential(nn.Conv2d(1, 3, 1),
                                   nn.Conv2d(3, 3, 3, groups=1),
                                   nn.Conv2d(3, 3, 1))
        self.conv2 = nn.Sequential(nn.Conv2d(3, 10, 1),
                                   nn.Conv2d(10, 10, 3, groups=10),
                                   nn.Conv2d(10, 10, 1))
        self.conv3 = nn.Sequential(nn.Conv2d(10, 10, 1),
                                   nn.Conv2d(10, 10, 3, groups=10),
                                   nn.Conv2d(10, 10, 1))
        self.conv4 = nn.Sequential(nn.Conv2d(10, 20, 1),
                                   nn.Conv2d(20, 20, 3, groups=20),
                                   nn.Conv2d(20, 20, 1))
        self.conv5 = nn.Sequential(nn.Conv2d(20, 30, 1),
                                   nn.Conv2d(30, 30, 3, groups=30),
                                   nn.Conv2d(30, 30, 1))
        self.conv6 = nn.Sequential(nn.Conv2d(30, 40, 1),
                                   nn.Conv2d(40, 40, 3, groups=40),
                                   nn.Conv2d(40, 40, 1))
        self.conv7 = nn.Sequential(nn.Conv2d(40, 10, 1),
                                   nn.Conv2d(10, 10, 3, groups=10),
                                   nn.Conv2d(10, 10, 1))
        self.fc1 = nn.Linear(10 * 14 * 14, 50)
        self.fc2 = nn.Linear(50, 10)
    
    def forward(self, x):
        # print("Before Calc")
        x = F.relu(self.conv1(x))
        # print("After Conv1", x.size())
        x = F.relu(self.conv2(x))
        # print("After Conv2", x.size())
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))
        x = F.relu(self.conv6(x))
        x = F.relu(self.conv7(x))
        # print("After Conv3", x.size())
        x = x.view(-1, 10 * 14 * 14)
        # print("After dim reduction", x.size())
        x = F.relu(self.fc1(x))
        # print("After fc1", x.size())
        x = self.fc2(x)
        # print("After fc2", x.size())
        return x

sepcnn = SepCNN()
output = sepcnn(torch.randn(1, 1, 28, 28))

In [4]:
training_data = FashionMNIST(
    root="data",
    train=True,
    download=False,
    transform=ToTensor()
)

test_data = FashionMNIST(
    root="data",
    train=False,
    download=False,
    transform=ToTensor()
)

In [5]:
batch_size = 64

# 데이터로더(dataloader) 객체의 각 요소는 64개의 특징(feature)과 정답(label)을 묶음(batch)으로 반환
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y torch.Size([64]) torch.int64


In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

from copy import deepcopy

# test_cnn = deepcopy(cnn)
# test_sep = deepcopy(sepcnn)

# test_cnn = test_cnn.to(device)
# test_sep = test_sep.to(device)
cnn = cnn.to(device)

Using cuda device


In [11]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(cnn.parameters(), lr=0.0015)

In [12]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        # 예측 오류 계산
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [13]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [14]:
epochs = 30
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, cnn, loss_fn, optimizer)
    test(test_dataloader, cnn, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 0.461265  [    0/60000]
loss: 0.551017  [ 6400/60000]
loss: 0.324204  [12800/60000]
loss: 0.597105  [19200/60000]
loss: 0.534846  [25600/60000]
loss: 0.601369  [32000/60000]
loss: 0.544977  [38400/60000]
loss: 0.574193  [44800/60000]
loss: 0.475933  [51200/60000]
loss: 0.510439  [57600/60000]
Test Error: 
 Accuracy: 79.0%, Avg loss: 0.564163 

Epoch 2
-------------------------------
loss: 0.464834  [    0/60000]
loss: 0.542373  [ 6400/60000]
loss: 0.314229  [12800/60000]
loss: 0.579464  [19200/60000]
loss: 0.520154  [25600/60000]
loss: 0.596851  [32000/60000]
loss: 0.526486  [38400/60000]
loss: 0.579632  [44800/60000]
loss: 0.461693  [51200/60000]
loss: 0.500348  [57600/60000]
Test Error: 
 Accuracy: 79.3%, Avg loss: 0.557385 

Epoch 3
-------------------------------
loss: 0.453895  [    0/60000]
loss: 0.533684  [ 6400/60000]
loss: 0.304388  [12800/60000]
loss: 0.564646  [19200/60000]
loss: 0.513313  [25600/60000]
loss: 0.591422  [32000/600